<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')



In [ ]:
!python -m pip install --upgrade pip
!pip install pyLDAvis
# import pyLDAvis
import pyLDAvis.gensim_models # don't skip this

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa


from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

In [ ]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(5)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2002 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2010 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchu

In [ ]:
bug_report["Priority"].value_counts()

In [ ]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 79.4 ms, sys: 1.03 ms, total: 80.5 ms
Wall time: 80.9 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [ ]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 481 ms, sys: 3.95 ms, total: 485 ms
Wall time: 486 ms


In [ ]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 15.2 s, sys: 113 ms, total: 15.3 s
Wall time: 15.3 s


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [ ]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 9.26 ms, sys: 1.98 ms, total: 11.2 ms
Wall time: 11.7 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,25,P3,28126.0,cvs ui need vcm prefs default repo connection,it would be helpful if there was a notion of default repository connection so that when you connected a project you wouldnt have to walk through the list everytime not sure how this goes into prefs if its never been created before maybe when you create them you can mark one as the default at a minimum having a default repository type cvscc would be helpful which then the pulldown in the repo connection wizard has a default notes


In [ ]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(5)

CPU times: user 31.8 ms, sys: 2.04 ms, total: 33.9 ms
Wall time: 37.9 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...
1,2,P5,NaN,opening repository resources doesnt honor type,opening repository resource always open the default text editor and doesnt honor any mapping between resource types and editors as a result it is not possible to view the contents of an image gif file in a sensible way notes eg am agreed that this is needed eventually but not critical for june there should also be a java outliner when editing a java resource from the repository moving to vcm km pm in general you cannot open a specialized editor on a remote resource because they can sometimes be dependent on other resources and expect these to be local for example an html page which references a graphic the html editor is going to try to look for the graphic to display it you cant be sure that the editor was written to accomodate remote or missing resources we discussed this some time ago with dave t and felt it was safer to just use vanilla editors for now km pm see also itpvcmall repositories view all file types open to the text editor
2,3,P5,NaN,sync does not indicate deletion,km pm this pr about the deletion indicator in the sync viewer being too subtle this would be even more true for someone with vision problems ne has good suggestion of using the label of the compare text viewer we may also consider a better icon on to the pr delete file from project synchronize there is no indication that the file has been deleted from the workspace it looks the same as if the file is in the workspace but is empty notes km pm it should appear as a rightfacing arrow in the sync view sent screen cap to ne requesting clarification ne am the in the arrow is clearer when there are other arrows present too the and change ones otherwise its easy to miss the distinction especially when the users attention is focussed on the text compare panes id expect the workspace file label to indicate that it has been deleted the repository file label does this if i remember correctly when it has been deleted from the repository
3,4,P5,NaN,need better error message if catching up over readonly resource,become synchronized with some project in a repository use a different eclipse to make a change to a file resource within this project and release it to the repository in the original eclipse mark this file resource as being readonly select it rightclick properties change ok select the file resource rightclick team synchronize with stream in the subsequent comparison view select the file resource rightclick catchup since it has been marked as readonly there are inevitable problems however the error dialog that is shown does not offer much assistance an io error occurred io error notes
4,5,P3,NaN,isharingmanager sharing api inconsistent,for gettingsetting the managed state of a resource the methods are ismanagedresource and manageresource for gettingsetting the ignore state the methods are getignoredresource setignoredresource these should be

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from time import time
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=2) ## Corpus is in English
X = vectorizer.fit_transform(master_reports.Description)
print(X.shape)
# import matplotlib.pyplot as plt

# Sum_of_squared_distances = []
# K = range(2,10)
# for k in K:
#    km = KMeans(n_clusters=k, max_iter=200, n_init=10)
#    km = km.fit(X)
#    Sum_of_squared_distances.append(km.inertia_)
# plt.plot(K, Sum_of_squared_distances, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Sum_of_squared_distances')
# plt.title('Elbow Method For Optimal k')
# plt.show()



km = KMeans(n_clusters=10, init='k-means++', max_iter=100)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))

(70629, 59977)
done in 48.328s


In [67]:
centroids = km.cluster_centers_.argsort()[:, ::-1] ## Indices of largest centroids' entries in descending order
terms = vectorizer.get_feature_names()
for i in range(10):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :100]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: bug build code new use method need does like api text using patch class set used test swt created null change problem line currently fix add support id plugin following work just ui return run user example extension case time changes instead files make value error way string windows attached attachment tests message default dont eclipse public action table image ant added doesnt list source widget cvs version selection java type javadoc file tree platform needs im create thread debug current content possible key think missing called check point update control help changed resource exception remove view element look int
Cluster 1: dialog page button menu open select click search help window text perspective preference build tab new launch preferences does item press focus set selected buttons workbench list user right run font eclipse steps field use java type windows configuration context box ok mouse default reproduce using selection bar change problem items key view close 

In [71]:
Y = vectorizer.transform(["opening repository resource always open the default text editor and doesnt honor any mapping between resource types and editors as a result it is not possible"])
prediction = km.predict(Y)
print(prediction)

[3]


In [ ]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

In [ ]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


In [ ]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

In [ ]:
file = open('drive/My Drive/dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [ ]:
f= open('drive/My Drive/bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bag_of_words_corpus, f)

In [ ]:
lda_model.save('drive/My Drive/lda_model.model')

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

In [ ]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."


bow_vector = topic=lda_model[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
%%time


for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))


In [ ]:
%%time

for c in range(10):
  exec('topic_{} = pd.read_csv("topic_{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

CPU times: user 11 s, sys: 354 ms, total: 11.3 s
Wall time: 11.3 s


In [ ]:
%%time

for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x = topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

CPU times: user 1.55 s, sys: 4.97 ms, total: 1.56 s
Wall time: 1.57 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))
  

In [ ]:
!pip install glove-python-binary

In [ ]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))    # Path to a corpus file
  vector_size = 100                           # Dimensionality of the word vectors
  w = 6                                       #Maximum distance between the current and predicted word within a sentence
  min_count = 5                               # Ignores all words with total frequency lower than this.
  CBoW = 0                                    # Training algorithm: 1 for skip-gram; otherwise CBOW(The model tries to predict the target word by trying to understand the context of the surrounding words.).
  iterations = 100

  exec('w2vmodel{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  exec('path = get_tmpfile("word2vec{}.model")'.format(cluster))
  exec('w2vmodel{}.save("word2vec{}.model")'.format(cluster, cluster))

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("ftmodel{}.model")'.format(cluster))
  exec('ftmodel{}.save("ftmodel{}.model")'.format(cluster, cluster))

In [ ]:
%%time

for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
  exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("glove{}.model")'.format(cluster))
  exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)
from glove import Glove, Corpus

In [ ]:
f = open('drive/My Drive/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('drive/My Drive/dictionary.pickle', 'rb')
dictionary=pickle.load(file)

# later on, load trained model from file
lda_model =  models.LdaModel.load('drive/My Drive/lda_model.model')

In [ ]:
# for c in range(10):
#   exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
#   exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
#   exec("topic_{}['Description'] = topic_{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

In [ ]:
test = duplicated_reports
test.head()

In [ ]:
for mod in range(10):
  #import all the trained Word2Vec models
  exec('w2vmodel{} = Word2Vec.load("word2vec{}.model")'.format(mod, mod))

  #import all the trained FastText models
  exec('ftmodel{} = FastText.load("ftmodel{}.model")'.format(mod, mod))

  #import all the trained GloVe models
  exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [ ]:
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x = lda_model[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x = topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    return sim


In [ ]:
# no_of_test_samples= int(200)
vec_acc=[]
t1 = time.time()
for i in range(len(test)):
  sample = test.Description[i]
  n = 3
  max_cluster = sim_with_clusters_lda_topn(sample, n)

  for max in max_cluster:
    exec('cluster = topic_{}'.format(max))              #The predicted cluster
    exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
    exec('model2 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
    exec('sent = topic_{}.Description'.format(max)) 
  
